In [ ]:
import kagglehub
kagglehub.login()


In [ ]:
sushantgangurde0907_tuning_tiny_bert_linear_learning_rate_e30_f1_88_pytorch_default_2_path = kagglehub.model_download('sushantgangurde0907/tuning-tiny-bert_linear_learning_rate_e30_f1_88/PyTorch/default/2')

print('Data source import complete.')


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:

MODEL_PATH = "/kaggle/input/tuning-tiny-bert_linear_learning_rate_e30_f1_88/pytorch/default/1"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LABEL_MAP = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
REVERSE_LABEL_MAP = {v: k for k, v in LABEL_MAP.items()}

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:

try:
    print(f"Loading model from: {MODEL_PATH}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

    model.to(DEVICE)
    model.eval()
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

def predict_sentiment(text: str):
    """
    Takes a string of text and returns the predicted sentiment and confidence score.
    """
    if not text.strip():
        return "Invalid Input", 0.0

    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128, 
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt', 
    )

    input_ids = inputs['input_ids'].to(DEVICE)
    attention_mask = inputs['attention_mask'].to(DEVICE)

    with torch.no_grad(): 
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits

    prediction_idx = torch.argmax(logits, dim=1).item()

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    confidence = probabilities.max().item()

    predicted_label = REVERSE_LABEL_MAP[prediction_idx]

    return predicted_label, confidence


Loading model from: /kaggle/input/tuning-tiny-bert_linear_learning_rate_e30_f1_88/pytorch/default/1


2025-08-30 06:22:00.898735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756534921.086427      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756534921.147982      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model and tokenizer loaded successfully.


In [ ]:


# --- Main Execution ---
if __name__ == "__main__":
    print("\n--- Sentiment Analysis Inference ---")

    test_texts = [
        "The new update is fantastic and runs so smoothly!",
        "I am so angry, my package never arrived.",
        "The movie was just okay, nothing special about it.",
        "This is the worst customer service I have ever experienced.",
        "I'm feeling neither happy nor sad about the news.",
    ]

    for text in test_texts:
        sentiment, conf = predict_sentiment(text)
        print(f"\nText:       '{text}'")
        print(f"Sentiment:  {sentiment} (Confidence: {conf:.4f})")



--- Sentiment Analysis Inference ---

Text:       'The new update is fantastic and runs so smoothly!'
Sentiment:  Positive (Confidence: 0.9282)

Text:       'I am so angry, my package never arrived.'
Sentiment:  Negative (Confidence: 0.9639)

Text:       'The movie was just okay, nothing special about it.'
Sentiment:  Neutral (Confidence: 0.8169)

Text:       'This is the worst customer service I have ever experienced.'
Sentiment:  Negative (Confidence: 0.9062)

Text:       'I'm feeling neither happy nor sad about the news.'
Sentiment:  Negative (Confidence: 0.5409)


In [ ]:
import torch
import os
import zipfile
import shutil
from transformers import AutoTokenizer, AutoModelForSequenceClassification
PYTORCH_MODEL_PATH = "/kaggle/input/tuning-tiny-bert_linear_learning_rate_e30_f1_88/pytorch/default/2"
OUTPUT_PACKAGE_DIR = "./sentiment_model_js_package4"



ZIP_FILENAME = "sentiment_model_js4.zip"

print(f"--- Preparing Model Package from: {PYTORCH_MODEL_PATH} ---")

if not os.path.isdir(PYTORCH_MODEL_PATH):
    print(f"\n[ERROR] The directory '{PYTORCH_MODEL_PATH}' was not found.")
    print("Please make sure this path points to your saved model directory.")
    exit()

os.makedirs(OUTPUT_PACKAGE_DIR, exist_ok=True)
try:
    print("\nStep 1: Loading PyTorch model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(PYTORCH_MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(PYTORCH_MODEL_PATH)
    model.eval() # Set to evaluat
    print("PyTorch model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

onnx_output_path = os.path.join(OUTPUT_PACKAGE_DIR, "model.onnx")

try:
    print(f"\nStep 2: Exporting model to ONNX format at: {onnx_output_path}")
    dummy_input_ids = torch.randint(0, tokenizer.vocab_size, (1, 128))
    dummy_attention_mask = torch.ones(1, 128, dtype=torch.long)
    dummy_input = (dummy_input_ids, dummy_attention_mask)

    torch.onnx.export(
        model,
        dummy_input,
        onnx_output_path,
        input_names=['input_ids', 'attention_mask'],
        output_names=['logits'],
        dynamic_axes={
            'input_ids': {0: 'batch_size'},
            'attention_mask': {0: 'batch_size'},
            'logits': {0: 'batch_size'}
        },
        opset_version=14 
    )
    print("ONNX export successful.")
except Exception as e:
    print(f"Error during ONNX export: {e}")
    exit()

print(f"\nStep 3: Copying configuration files to '{OUTPUT_PACKAGE_DIR}'...")
try:
    required_files = [
        "config.json",
        "tokenizer_config.json",
        "vocab.txt",
        "special_tokens_map.json"
    ]
    for filename in required_files:
        source_path = os.path.join(PYTORCH_MODEL_PATH, filename)
        destination_path = os.path.join(OUTPUT_PACKAGE_DIR, filename)
        if os.path.exists(source_path):
            print(f"  - Copying '{filename}'...")
            shutil.copy(source_path, destination_path)
        else:
            print(f"  - WARNING: Expected file '{filename}' not found in source directory. Skipping.")

except Exception as e:
    print(f"Error copying configuration files: {e}")
    exit()

print(f"\nStep 4: Creating complete zip archive: {ZIP_FILENAME}")
try:
    with zipfile.ZipFile(ZIP_FILENAME, 'w', zipfile.ZIP_DEFLATED) as zf:
        print("Packaging the following files:")
        for filename in os.listdir(OUTPUT_PACKAGE_DIR):
            file_path = os.path.join(OUTPUT_PACKAGE_DIR, filename)
            print(f"  - Adding '{filename}'...")
            zf.write(file_path, arcname=filename)

    print("\nZip file created successfully.")
    print(f"\n--- Conversion Complete ---")
    print(f"Your JavaScript-ready model is in '{ZIP_FILENAME}'.")
    print("Unzip this file in your web project directory.")

except Exception as e:
    print(f"Error creating zip file: {e}")



--- Preparing Model Package from: /kaggle/input/tuning-tiny-bert_linear_learning_rate_e30_f1_88/pytorch/default/2 ---

Step 1: Loading PyTorch model and tokenizer...


2025-08-30 07:24:38.347474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756538678.528689      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756538678.580640      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch model loaded successfully.

Step 2: Exporting model to ONNX format at: ./sentiment_model_js_package4/model.onnx
ONNX export successful.

Step 3: Copying configuration files to './sentiment_model_js_package4'...
  - Copying 'config.json'...
  - Copying 'tokenizer_config.json'...
  - Copying 'vocab.txt'...
  - Copying 'special_tokens_map.json'...

Step 4: Creating complete zip archive: sentiment_model_js4.zip
Packaging the following files:
  - Adding 'special_tokens_map.json'...
  - Adding 'config.json'...
  - Adding 'vocab.txt'...
  - Adding 'model.onnx'...
  - Adding 'tokenizer_config.json'...

Zip file created successfully.

--- Conversion Complete ---
Your JavaScript-ready model is in 'sentiment_model_js4.zip'.
Unzip this file in your web project directory.
